# Fusion Retrieval: Combining Vector and Keyword Search

## Setting Up the Environment

In [3]:
#%pip install rank-bm25

In [22]:
import numpy as np
from rank_bm25 import BM25Okapi
import fitz
import re
import json
import time
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Document Processing Functions

In [25]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트 내용을 추출합니다.
    
    Args:
        pdf_path (str): PDF 파일 경로
        
    Returns:
        str: 추출된 텍스트 내용
    """
    print(f"{pdf_path}에서 텍스트 추출 중...")  # 현재 처리 중인 PDF 경로 출력
    pdf_document = fitz.open(pdf_path)  # PyMuPDF를 사용하여 PDF 파일 열기
    text = ""  # 추출된 텍스트를 저장할 빈 문자열 초기화
    
    # PDF의 각 페이지를 순회하며 텍스트 추출
    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]  # 페이지 객체 가져오기
        text += page.get_text()  # 해당 페이지에서 텍스트를 추출하고 누적
    
    return text  # 최종적으로 추출된 전체 텍스트 반환

In [26]:
def chunk_text(text, chunk_size=1000, chunk_overlap=200):
    """
    텍스트를 오버랩이 포함된 청크로 분할합니다.
    
    Args:
        text (str): 분할할 입력 텍스트
        chunk_size (int): 각 청크의 문자 수
        chunk_overlap (int): 청크 간 중첩 문자 수

    Returns:
        List[Dict]: 텍스트와 메타데이터를 포함한 청크 리스트
    """
    chunks = []  # 청크 데이터를 저장할 리스트 초기화

    # 지정된 크기와 오버랩에 따라 텍스트를 순회하며 청크 생성
    for i in range(0, len(text), chunk_size - chunk_overlap):
        chunk = text[i:i + chunk_size]  # 현재 위치에서 chunk_size만큼 자르기
        if chunk:  # 비어 있지 않은 청크만 추가
            chunk_data = {
                "text": chunk,  # 청크 텍스트
                "metadata": {
                    "start_char": i,               # 청크 시작 위치 (문자 인덱스)
                    "end_char": i + len(chunk)     # 청크 종료 위치 (문자 인덱스)
                }
            }
            chunks.append(chunk_data)  # 청크 데이터를 리스트에 추가

    print(f"{len(chunks)}개의 텍스트 청크가 생성되었습니다")  # 생성된 청크 수 출력
    return chunks  # 청크 리스트 반환

In [27]:
def clean_text(text):
    """
    텍스트에서 불필요한 공백 및 특수 문자를 제거하여 정리합니다.
    
    Args:
        text (str): 정리할 입력 텍스트

    Returns:
        str: 정리된 텍스트
    """
    # 여러 개의 공백 문자(스페이스, 줄바꿈, 탭 등)를 하나의 공백으로 치환
    text = re.sub(r'\s+', ' ', text)
    
    # OCR 오류로 생긴 이스케이프된 탭(\t)과 줄바꿈(\n) 문자를 공백으로 치환
    text = text.replace('\\t', ' ')
    text = text.replace('\\n', ' ')
    
    # 앞뒤 공백 제거 및 단어 사이 공백을 하나로 통일
    text = ' '.join(text.split())
    
    return text

## Creating Vector Store

In [28]:
def create_embeddings(texts, model="text-embedding-3-small"):
    """
    주어진 텍스트(들)에 대해 임베딩을 생성합니다.
    
    Args:
        texts (str 또는 List[str]): 입력 텍스트 또는 텍스트 리스트
        model (str): 사용할 임베딩 모델 이름
        
    Returns:
        List[List[float]] 또는 List[float]: 생성된 임베딩 벡터(들)
    """
    # 문자열과 리스트 둘 다 처리 가능하도록 통일
    input_texts = texts if isinstance(texts, list) else [texts]
    
    # API 호출 제한을 고려한 배치 처리
    batch_size = 100
    all_embeddings = []  # 전체 임베딩 결과 저장 리스트
    
    # 배치 단위로 입력 텍스트 처리
    for i in range(0, len(input_texts), batch_size):
        batch = input_texts[i:i + batch_size]  # 현재 배치 텍스트 가져오기
        
        # 해당 배치에 대해 임베딩 생성 요청
        response = client.embeddings.create(
            model=model,
            input=batch
        )
        
        # 응답에서 임베딩 벡터 추출
        batch_embeddings = [item.embedding for item in response.data]
        all_embeddings.extend(batch_embeddings)  # 전체 임베딩 리스트에 추가
    
    # 입력이 문자열 하나였다면, 첫 번째 결과만 반환
    if isinstance(texts, str):
        return all_embeddings[0]
    
    # 리스트 입력인 경우 전체 임베딩 리스트 반환
    return all_embeddings

In [29]:
class SimpleVectorStore:
    """
    NumPy를 사용한 간단한 벡터 저장소 구현입니다.
    """
    def __init__(self):
        self.vectors = []  # 임베딩 벡터를 저장할 리스트
        self.texts = []  # 텍스트 내용을 저장할 리스트
        self.metadata = []  # 메타데이터를 저장할 리스트

    def add_item(self, text, embedding, metadata=None):
        """
        단일 항목을 벡터 저장소에 추가합니다.
        
        Args:
            text (str): 텍스트 내용
            embedding (List[float]): 임베딩 벡터
            metadata (Dict, 선택): 추가 메타데이터
        """
        self.vectors.append(np.array(embedding))  # 임베딩 벡터 추가
        self.texts.append(text)  # 텍스트 추가
        self.metadata.append(metadata or {})  # 메타데이터 추가 (없으면 빈 딕셔너리)

    def add_items(self, items, embeddings):
        """
        여러 항목을 저장소에 일괄 추가합니다.
        
        Args:
            items (List[Dict]): 텍스트 및 메타데이터가 포함된 항목 리스트
            embeddings (List[List[float]]): 각 항목의 임베딩 벡터 리스트
        """
        for i, (item, embedding) in enumerate(zip(items, embeddings)):
            self.add_item(
                text=item["text"],  # 항목에서 텍스트 추출
                embedding=embedding,  # 해당 임베딩 사용
                metadata={**item.get("metadata", {}), "index": i}  # 기존 메타데이터에 인덱스 추가
            )

    def similarity_search_with_scores(self, query_embedding, k=5):
        """
        쿼리 임베딩과 가장 유사한 항목 k개를 유사도 점수와 함께 반환합니다.
        
        Args:
            query_embedding (List[float]): 쿼리 임베딩 벡터
            k (int): 반환할 유사 항목 수
            
        Returns:
            List[Dict]: 유사도 상위 k개의 항목 (텍스트, 메타데이터, 유사도 포함)
        """
        if not self.vectors:
            return []  # 벡터가 없으면 빈 리스트 반환

        query_vector = np.array(query_embedding)  # 쿼리 벡터를 NumPy 배열로 변환
        similarities = []

        # 코사인 유사도 계산
        for i, vector in enumerate(self.vectors):
            similarity = cosine_similarity([query_vector], [vector])[0][0]
            similarities.append((i, similarity))  # 인덱스와 유사도 저장

        # 유사도 기준 내림차순 정렬
        similarities.sort(key=lambda x: x[1], reverse=True)

        # 상위 k개 항목 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],  # 해당 텍스트
                "metadata": self.metadata[idx],  # 해당 메타데이터
                "similarity": float(score)  # 유사도 점수
            })

        return results

    def get_all_documents(self):
        """
        저장소에 저장된 모든 문서를 가져옵니다.
        
        Returns:
            List[Dict]: 텍스트와 메타데이터가 포함된 문서 리스트
        """
        return [{"text": text, "metadata": meta} for text, meta in zip(self.texts, self.metadata)]

## BM25 Implementation

In [30]:
def create_bm25_index(chunks):
    """
    주어진 텍스트 청크들로부터 BM25 인덱스를 생성합니다.
    
    Args:
        chunks (List[Dict]): 텍스트 청크 딕셔너리 리스트
        
    Returns:
        BM25Okapi: 생성된 BM25 인덱스 객체
    """
    # 각 청크에서 텍스트만 추출
    texts = [chunk["text"] for chunk in chunks]
    
    # 공백 기준으로 각 문서를 토큰화
    tokenized_docs = [text.split() for text in texts]
    
    # 토큰화된 문서를 이용해 BM25 인덱스 생성
    bm25 = BM25Okapi(tokenized_docs)
    
    # 생성된 문서 개수 출력
    print(f"BM25 인덱스에 {len(texts)}개의 문서가 포함되었습니다")
    
    return bm25

In [31]:
def bm25_search(bm25, chunks, query, k=5):
    """
    BM25 인덱스를 사용하여 쿼리에 대한 검색을 수행합니다.
    
    Args:
        bm25 (BM25Okapi): 생성된 BM25 인덱스 객체
        chunks (List[Dict]): 텍스트 청크 리스트
        query (str): 검색 쿼리 문자열
        k (int): 반환할 결과 수 (Top-k)

    Returns:
        List[Dict]: 상위 k개의 결과와 BM25 점수를 포함한 리스트
    """
    # 쿼리를 공백 기준으로 토큰화
    query_tokens = query.split()
    
    # 쿼리 토큰에 대한 각 문서의 BM25 점수 계산
    scores = bm25.get_scores(query_tokens)
    
    results = []  # 점수와 함께 결과 저장할 리스트 초기화

    # 각 점수와 해당 청크에 대해 순회
    for i, score in enumerate(scores):
        # 원본을 수정하지 않도록 메타데이터 복사
        metadata = chunks[i].get("metadata", {}).copy()
        # 청크 인덱스를 메타데이터에 추가
        metadata["index"] = i

        # 결과 딕셔너리에 텍스트, 메타데이터, 점수 포함
        results.append({
            "text": chunks[i]["text"],
            "metadata": metadata,
            "bm25_score": float(score)
        })
    
    # BM25 점수를 기준으로 결과 내림차순 정렬
    results.sort(key=lambda x: x["bm25_score"], reverse=True)
    
    # 상위 k개의 결과 반환
    return results[:k]

## Fusion Retrieval Function

In [32]:
def fusion_retrieval(query, chunks, vector_store, bm25_index, k=5, alpha=0.5):
    """
    벡터 기반 검색과 BM25 검색을 결합한 하이브리드 검색(fusion retrieval)을 수행합니다.
    
    Args:
        query (str): 사용자 쿼리 문자열
        chunks (List[Dict]): 원본 텍스트 청크 리스트
        vector_store (SimpleVectorStore): 벡터 저장소
        bm25_index (BM25Okapi): BM25 인덱스
        k (int): 반환할 결과 수
        alpha (float): 벡터 점수 가중치 (0~1), 나머지 (1-alpha)는 BM25 가중치
        
    Returns:
        List[Dict]: 결합 점수 기준 상위 k개 검색 결과
    """
    print(f"쿼리에 대한 하이브리드 검색 수행 중: {query}")
    
    epsilon = 1e-8  # 0으로 나누는 것을 방지하기 위한 아주 작은 수

    # 쿼리 임베딩 생성 및 벡터 검색 수행
    query_embedding = create_embeddings(query)
    vector_results = vector_store.similarity_search_with_scores(
        query_embedding, k=len(chunks)
    )

    # BM25 검색 수행
    bm25_results = bm25_search(bm25_index, chunks, query, k=len(chunks))

    # 결과 점수를 인덱스로 매핑 (index → score)
    vector_scores_dict = {r["metadata"]["index"]: r["similarity"] for r in vector_results}
    bm25_scores_dict = {r["metadata"]["index"]: r["bm25_score"] for r in bm25_results}

    # 모든 문서에 대해 점수 결합 준비
    all_docs = vector_store.get_all_documents()
    combined_results = []

    for i, doc in enumerate(all_docs):
        vector_score = vector_scores_dict.get(i, 0.0)  # 없으면 0.0
        bm25_score = bm25_scores_dict.get(i, 0.0)
        combined_results.append({
            "text": doc["text"],
            "metadata": doc["metadata"],
            "vector_score": vector_score,
            "bm25_score": bm25_score,
            "index": i
        })

    # 점수 배열로 변환 및 정규화 (0~1 사이로)
    vector_scores = np.array([doc["vector_score"] for doc in combined_results])
    bm25_scores = np.array([doc["bm25_score"] for doc in combined_results])

    norm_vector_scores = (vector_scores - np.min(vector_scores)) / (np.max(vector_scores) - np.min(vector_scores) + epsilon)
    norm_bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores) + epsilon)

    # 결합 점수 계산 (선형 가중 평균)
    combined_scores = alpha * norm_vector_scores + (1 - alpha) * norm_bm25_scores

    # 결합 점수를 결과에 추가
    for i, score in enumerate(combined_scores):
        combined_results[i]["combined_score"] = float(score)

    # 결합 점수 기준으로 내림차순 정렬
    combined_results.sort(key=lambda x: x["combined_score"], reverse=True)

    # 상위 k개 결과 반환
    top_results = combined_results[:k]

    print(f"하이브리드 검색으로 {len(top_results)}개의 문서를 반환함")
    return top_results

## Document Processing Pipeline

In [33]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    하이브리드 검색(fusion retrieval)을 위한 문서를 처리합니다.
    
    Args:
        pdf_path (str): PDF 파일 경로
        chunk_size (int): 각 청크의 문자 수
        chunk_overlap (int): 청크 간 중첩 문자 수

    Returns:
        Tuple[List[Dict], SimpleVectorStore, BM25Okapi]:
            - 텍스트 청크 리스트
            - 벡터 저장소(SimpleVectorStore)
            - BM25 인덱스(BM25Okapi)
    """
    # PDF 파일에서 텍스트 추출
    text = extract_text_from_pdf(pdf_path)
    
    # 추출된 텍스트를 정리 (공백, 특수문자 제거)
    cleaned_text = clean_text(text)
    
    # 텍스트를 오버랩 포함 청크로 분할
    chunks = chunk_text(cleaned_text, chunk_size, chunk_overlap)
    
    # 임베딩 생성을 위해 각 청크의 텍스트만 추출
    chunk_texts = [chunk["text"] for chunk in chunks]
    print("청크 임베딩 생성 중...")

    # 임베딩 생성
    embeddings = create_embeddings(chunk_texts)

    # 벡터 저장소 초기화 및 항목 추가
    vector_store = SimpleVectorStore()
    vector_store.add_items(chunks, embeddings)
    print(f"{len(chunks)}개의 항목이 벡터 저장소에 추가됨")

    # BM25 인덱스 생성
    bm25_index = create_bm25_index(chunks)

    # 청크, 벡터 저장소, BM25 인덱스 반환
    return chunks, vector_store, bm25_index

## Response Generation

In [34]:
def generate_response(query, context):
    """
    주어진 쿼리와 문맥을 바탕으로 AI 응답을 생성합니다.
    
    Args:
        query (str): 사용자 질문
        context (str): 검색된 문서에서 제공된 문맥 정보

    Returns반환값:
        str: 생성된 응답
    """
    # AI의 역할을 안내하는 시스템 프롬프트 정의
    system_prompt = """당신은 유용한 AI 어시스턴트입니다. 제공된 문맥을 바탕으로 사용자의 질문에 답변하세요. 
    문맥에 질문에 대한 충분한 정보가 없다면, 그 한계를 분명히 언급해 주세요."""

    # 사용자 프롬프트 구성 (문맥 + 질문)
    user_prompt = f"""문맥:
    {context}

    질문: {query}

    위의 문맥을 바탕으로 질문에 답변해 주세요."""

    # OpenAI API를 사용하여 응답 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # 사용할 모델 지정
        messages=[
            {"role": "system", "content": system_prompt},  # 시스템 메시지 (AI 가이드)
            {"role": "user", "content": user_prompt}  # 사용자 메시지 (문맥 + 질문)
        ],
        temperature=0.1  # 응답의 일관성을 높이기 위한 낮은 창의성 설정
    )
    
    # 생성된 응답 반환
    return response.choices[0].message.content

## Main Retrieval Function

In [35]:
def answer_with_fusion_rag(query, chunks, vector_store, bm25_index, k=5, alpha=0.5):
    """
    Fusion RAG 기법을 사용하여 쿼리에 응답합니다.
    
    Args:
        query (str): 사용자 질문
        chunks (List[Dict]): 텍스트 청크 리스트
        vector_store (SimpleVectorStore): 벡터 저장소
        bm25_index (BM25Okapi): BM25 인덱스
        k (int): 검색할 문서 개수
        alpha (float): 벡터 점수의 가중치 (0~1)

    Returns:
        Dict: 쿼리, 검색된 문서, 생성된 응답을 포함한 결과 딕셔너리
    """
    # Fusion retrieval 방식으로 관련 문서 검색
    retrieved_docs = fusion_retrieval(query, chunks, vector_store, bm25_index, k=k, alpha=alpha)

    # 검색된 문서들의 텍스트를 구분선으로 연결하여 문맥 구성
    context = "\n\n---\n\n".join([doc["text"] for doc in retrieved_docs])

    # 문맥과 쿼리를 기반으로 AI 응답 생성
    response = generate_response(query, context)

    # 쿼리, 검색 문서, 생성 응답을 딕셔너리로 반환
    return {
        "query": query,
        "retrieved_documents": retrieved_docs,
        "response": response
    }

## Comparing Retrieval Methods

In [36]:
def vector_only_rag(query, vector_store, k=5):
    """
    벡터 기반 RAG만을 사용하여 쿼리에 응답합니다.
    
    Args:
        query (str): 사용자 질문
        vector_store (SimpleVectorStore): 벡터 저장소
        k (int): 검색할 문서 개수

    Returns:
        Dict: 쿼리, 검색 문서, 생성된 응답을 포함한 결과 딕셔너리
    """
    # 쿼리 임베딩 생성
    query_embedding = create_embeddings(query)
    
    # 벡터 유사도 기반으로 관련 문서 검색
    retrieved_docs = vector_store.similarity_search_with_scores(query_embedding, k=k)
    
    # 검색된 문서들의 텍스트를 구분선으로 연결하여 문맥 구성
    context = "\n\n---\n\n".join([doc["text"] for doc in retrieved_docs])
    
    # 문맥을 기반으로 쿼리에 대한 응답 생성
    response = generate_response(query, context)
    
    # 쿼리, 검색 문서, 응답을 딕셔너리 형태로 반환
    return {
        "query": query,
        "retrieved_documents": retrieved_docs,
        "response": response
    }

In [37]:
def bm25_only_rag(query, chunks, bm25_index, k=5):
    """
    BM25 기반 RAG만을 사용하여 쿼리에 응답합니다.

    Args:
        query (str): 사용자 질문
        chunks (List[Dict]): 텍스트 조각들
        bm25_index (BM25Okapi): BM25 인덱스
        k (int): 검색할 문서 개수

    Returns:
        Dict: 쿼리 결과
    """
    # BM25 검색을 사용하여 문서를 검색합니다
    retrieved_docs = bm25_search(bm25_index, chunks, query, k=k)
    
    # 검색된 문서의 텍스트를 구분자와 함께 연결하여 context를 생성합니다
    context = "\n\n---\n\n".join([doc["text"] for doc in retrieved_docs])
    
    # 쿼리와 구성된 context를 기반으로 응답을 생성합니다
    response = generate_response(query, context)
    
    # 쿼리, 검색된 문서들, 생성된 응답을 반환합니다
    return {
        "query": query,
        "retrieved_documents": retrieved_docs,
        "response": response
    }

## Evaluation Functions

In [38]:
def compare_retrieval_methods(query, chunks, vector_store, bm25_index, k=5, alpha=0.5, reference_answer=None):
    """
    하나의 쿼리에 대해 다양한 검색(RAG) 방법을 비교합니다.

    Args:
        query (str): 사용자 질문
        chunks (List[Dict]): 텍스트 조각들
        vector_store (SimpleVectorStore): 벡터 저장소
        bm25_index (BM25Okapi): BM25 인덱스
        k (int): 검색할 문서 개수
        alpha (float): 벡터 기반 점수의 가중치 (결합 검색 시 사용)
        reference_answer (str, optional): 정답 비교용 기준 응답

    Returns:
        Dict: 검색 방식별 결과 및 비교 내용
    """
    print(f"\n=== 쿼리에 대한 검색 방식 비교: {query} ===\n")
    
    # 벡터 기반 RAG 실행
    print("\n[1] 벡터 기반 RAG 실행 중...")
    vector_result = vector_only_rag(query, vector_store, k)
    
    # BM25 기반 RAG 실행
    print("\n[2] BM25 기반 RAG 실행 중...")
    bm25_result = bm25_only_rag(query, chunks, bm25_index, k)
    
    # 결합 기반(fusion) RAG 실행
    print("\n[3] 결합 기반 RAG 실행 중...")
    fusion_result = answer_with_fusion_rag(query, chunks, vector_store, bm25_index, k, alpha)
    
    # 서로 다른 검색 방식의 응답 비교
    print("\n[4] 응답 비교 중...")
    comparison = evaluate_responses(
        query, 
        vector_result["response"], 
        bm25_result["response"], 
        fusion_result["response"],
        reference_answer
    )
    
    # 비교 결과 반환
    return {
        "query": query,
        "vector_result": vector_result,
        "bm25_result": bm25_result,
        "fusion_result": fusion_result,
        "comparison": comparison
    }

In [39]:
def evaluate_responses(query, vector_response, bm25_response, fusion_response, reference_answer=None):
    """
    서로 다른 검색 방식에서 생성된 응답들을 평가합니다.

    Args:
        query (str): 사용자 질문
        vector_response (str): 벡터 기반 RAG 응답
        bm25_response (str): BM25 기반 RAG 응답
        fusion_response (str): 결합 기반 RAG 응답
        reference_answer (str, optional): 기준이 되는 정답 (선택 사항)

    Returns:
        str: 응답 평가 결과
    """
    # 평가를 위한 시스템 프롬프트: 평가자의 역할과 평가 기준 안내
    system_prompt = """당신은 RAG 시스템의 전문가 평가자입니다. 아래의 세 가지 검색 방식에서 생성된 응답을 비교해 주세요:
    1. 벡터 기반 검색: 의미적 유사성을 기반으로 문서를 검색합니다
    2. BM25 키워드 기반 검색: 키워드 일치를 기반으로 문서를 검색합니다
    3. 결합 기반 검색: 벡터 기반과 키워드 기반을 모두 결합하여 검색합니다

    다음 기준에 따라 응답을 평가해 주세요:
    - 질문과의 관련성
    - 사실에 기반한 정확성
    - 정보의 포괄성
    - 명확성 및 일관성
    """

    # 사용자 프롬프트: 질문과 세 가지 응답을 포함
    user_prompt = f"""질문: {query}

    벡터 기반 응답:
    {vector_response}

    BM25 키워드 기반 응답:
    {bm25_response}

    결합 기반 응답:
    {fusion_response}
    """

    # 기준 응답이 주어진 경우, 프롬프트에 추가
    if reference_answer:
        user_prompt += f"""
            기준 응답:
            {reference_answer}
        """

    # 평가자에게 상세 비교를 요청하는 문장 추가
    user_prompt += """
    위 세 응답에 대해 상세한 비교를 작성해 주세요. 이 쿼리에 가장 잘 대응한 방식은 무엇이며 그 이유는 무엇인가요?
    각각의 응답 방식의 장점과 단점을 구체적으로 설명해 주세요.
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",  # 사용할 모델 지정
        messages=[
            {"role": "system", "content": system_prompt},  # 시스템 메시지: 평가자 가이드
            {"role": "user", "content": user_prompt}  # 사용자 메시지: 질문과 응답들
        ],
        temperature=0  # 출력의 일관성을 위해 온도 설정
    )
    
    # 생성된 평가 결과 반환
    return response.choices[0].message.content

## Complete Evaluation Pipeline

In [40]:
def evaluate_fusion_retrieval(pdf_path, test_queries, reference_answers=None, k=5, alpha=0.5):
    """
    결합 기반 검색(fusion retrieval)을 다른 검색 방식과 비교하여 평가합니다.

    Args:
        pdf_path (str): PDF 파일 경로
        test_queries (List[str]): 테스트용 쿼리 목록
        reference_answers (List[str], optional): 기준 응답 목록 (선택 사항)
        k (int): 검색할 문서 개수
        alpha (float): 결합 검색 시 벡터 점수에 부여할 가중치

    Returns:
        Dict: 쿼리별 평가 결과 및 전체 분석 결과
    """
    print("=== 결합 기반 검색 평가 시작 ===\n")
    
    # 문서를 처리하여 텍스트 추출, 청크 생성, 벡터/BM25 인덱스 구축
    chunks, vector_store, bm25_index = process_document(pdf_path)
    
    # 쿼리별 평가 결과를 저장할 리스트 초기화
    results = []
    
    # 각 테스트 쿼리에 대해 반복 평가 수행
    for i, query in enumerate(test_queries):
        print(f"\n\n=== 쿼리 평가 중: {i+1}/{len(test_queries)} ===")
        print(f"질문: {query}")
        
        # 기준 응답이 있는 경우 가져오기
        reference = None
        if reference_answers and i < len(reference_answers):
            reference = reference_answers[i]
        
        # 현재 쿼리에 대해 검색 방식 비교 실행
        comparison = compare_retrieval_methods(
            query, 
            chunks, 
            vector_store, 
            bm25_index, 
            k=k, 
            alpha=alpha,
            reference_answer=reference
        )
        
        # 비교 결과를 리스트에 추가
        results.append(comparison)
        
        # 각 검색 방식의 응답 출력
        print("\n=== 벡터 기반 응답 ===")
        print(comparison["vector_result"]["response"])
        
        print("\n=== BM25 기반 응답 ===")
        print(comparison["bm25_result"]["response"])
        
        print("\n=== 결합 기반 응답 ===")
        print(comparison["fusion_result"]["response"])
        
        print("\n=== 응답 비교 결과 ===")
        print(comparison["comparison"])
    
    # 전체 쿼리에 대한 결합 검색 성능 분석 생성
    overall_analysis = generate_overall_analysis(results)
    
    # 전체 결과 및 분석 반환
    return {
        "results": results,
        "overall_analysis": overall_analysis
    }

In [41]:
def generate_overall_analysis(results):
    """
    결합 기반 검색(fusion retrieval)에 대한 전체 분석을 생성합니다.

    Args:
        results (List[Dict]): 각 쿼리별 평가 결과 목록

    Returns:
        str: 전체 분석 결과
    """
    # 평가 프로세스를 안내하는 시스템 프롬프트
    system_prompt = """당신은 정보 검색 시스템 평가 전문가입니다. 
    여러 테스트 쿼리를 기반으로 다음 세 가지 검색 방식을 비교한 전체 분석을 작성해 주세요:
    1. 벡터 기반 검색 (의미 유사도 기반)
    2. BM25 키워드 검색 (키워드 일치 기반)
    3. 결합 기반 검색 (두 가지 방식의 결합)

    다음 항목에 집중해 주세요:
    1. 각각의 방식이 가장 효과적인 쿼리 유형
    2. 각 방식의 전반적인 강점과 약점
    3. 결합 검색이 어떻게 절충점을 조율하는지
    4. 각 방식을 언제 사용하는 것이 적절한지에 대한 권장 사항
    """

    # 각 쿼리별 평가 내용을 요약한 문자열 생성
    evaluations_summary = ""
    for i, result in enumerate(results):
        evaluations_summary += f"쿼리 {i+1}: {result['query']}\n"
        evaluations_summary += f"비교 요약: {result['comparison'][:200]}...\n\n"

    # 사용자 프롬프트: 쿼리별 평가 요약 포함
    user_prompt = f"""{len(results)}개의 쿼리에 대해 다양한 검색 방식을 평가한 결과를 바탕으로, 
    벡터 기반, BM25 기반, 결합 기반 검색 방식을 전반적으로 비교 분석해 주세요:

    {evaluations_summary}

    특히 결합 검색이 어떤 상황에서 개별 방식보다 효과적인지, 
    그리고 각 방식의 사용 권장 조건을 포함한 종합적인 분석을 작성해 주세요.
    """

    # LLaMA 모델을 사용하여 전체 분석 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # 생성된 분석 결과 반환
    return response.choices[0].message.content

## Evaluating Fusion Retrieval

In [42]:
# PDF 문서 경로
# 지식 검색 테스트를 위한 AI 정보가 포함된 PDF 문서 경로
pdf_path = "dataset/AI_Understanding.pdf"

# 하나의 AI 관련 테스트 쿼리 정의
test_queries = [
    "자연어 처리에서 트랜스포머 모델의 주요 응용 분야는 무엇인가요?"  # AI 관련 쿼리
]

# (선택 사항) 기준이 되는 정답
reference_answers = [
    "트랜스포머 모델은 기계 번역, 텍스트 요약, 질문 답변, 감정 분석, 텍스트 생성 등의 애플리케이션을 통해 자연어 처리에 혁명을 일으켰습니다. 텍스트의 장거리 종속성을 포착하는 데 탁월하며 BERT, GPT, T5와 같은 모델의 기반이 되었습니다.",
]
# 검색 설정값 정의
k = 5  # 검색할 문서 개수
alpha = 0.5  # 벡터 점수 가중치 (0.5이면 벡터와 BM25 동일 비중)

# 평가 실행
evaluation_results = evaluate_fusion_retrieval(
    pdf_path=pdf_path,
    test_queries=test_queries,
    reference_answers=reference_answers,
    k=k,
    alpha=alpha
)

# 전체 분석 결과 출력
print("\n\n***전체 평가 요약***\n")
print(evaluation_results["overall_analysis"])

=== 결합 기반 검색 평가 시작 ===

dataset/AI_Understanding.pdf에서 텍스트 추출 중...
21개의 텍스트 청크가 생성되었습니다
청크 임베딩 생성 중...
21개의 항목이 벡터 저장소에 추가됨
BM25 인덱스에 21개의 문서가 포함되었습니다


=== 쿼리 평가 중: 1/1 ===
질문: 자연어 처리에서 트랜스포머 모델의 주요 응용 분야는 무엇인가요?

=== 쿼리에 대한 검색 방식 비교: 자연어 처리에서 트랜스포머 모델의 주요 응용 분야는 무엇인가요? ===


[1] 벡터 기반 RAG 실행 중...

[2] BM25 기반 RAG 실행 중...

[3] 결합 기반 RAG 실행 중...
쿼리에 대한 하이브리드 검색 수행 중: 자연어 처리에서 트랜스포머 모델의 주요 응용 분야는 무엇인가요?
하이브리드 검색으로 5개의 문서를 반환함

[4] 응답 비교 중...

=== 벡터 기반 응답 ===
제공된 문맥에서는 자연어 처리(NLP)와 관련된 트랜스포머 모델의 주요 응용 분야에 대한 구체적인 언급이 없습니다. 그러나 일반적으로 트랜스포머 모델은 기계 번역, 텍스트 요약, 감정 분석, 질문 응답 시스템 등 다양한 NLP 작업에 널리 사용됩니다. 이러한 정보는 문맥에 포함되어 있지 않으므로, 트랜스포머 모델의 응용 분야에 대한 구체적인 내용을 제공할 수는 없습니다.

=== BM25 기반 응답 ===
제공된 문맥에서는 자연어 처리(NLP)와 관련된 트랜스포머 모델의 주요 응용 분야에 대한 구체적인 정보가 포함되어 있지 않습니다. 그러나 일반적으로 트랜스포머 모델은 기계 번역, 텍스트 요약, 감정 분석, 질문 응답 시스템 등 다양한 NLP 작업에 널리 사용됩니다. 이러한 모델은 특히 문맥을 이해하고 긴 문장을 처리하는 데 강력한 성능을 발휘합니다. 추가적인 정보가 필요하다면 다른 자료를 참조하시기 바랍니다.

=== 결합 기반 응답 ===
제공된 문맥에는 자연어 처리(NLP)에서 트랜스포머 모델의 주요 응용 분야에 대한 정보가 포함되어 있지 않